# Problem 3

In [1]:
"""
5x5 Gridworld — Value Iteration (Synchronous vs In-Place)
---------------------------------------------------------
This single cell:
1) Defines the grid, rewards, discount, and transition function.
2) Implements two Value Iteration variants:
     - Synchronous (uses a copy of V_k to build V_{k+1})
     - In-Place / Asynchronous (updates V in place, using freshest values)
3) Extracts the greedy policy from the converged value function.
4) Prints the rewards, optimal values, optimal policies, and a runtime summary.
Everything uses plain ASCII so it renders well in any notebook/editor.
"""

import numpy as np
from time import perf_counter

# -------------------------------
# 1) PROBLEM SETUP
# -------------------------------

# Grid size and discount factor
n = 5            # 5x5 grid
gamma = 0.9      # discount factor

# Rewards:
# +10 at the terminal goal (4,4), -5 on "grey" cells, and -1 everywhere else.
R = -1 * np.ones((n, n))
grey = [(2, 2), (3, 0), (0, 4)]  # (row, col) positions of bad (grey) cells
for r, c in grey:
    R[r, c] = -5
goal = (4, 4)
R[goal] = 10

# Terminal set: reaching the goal ends the episode (absorbing)
terminal = {goal}

# Actions encoded as (delta_row, delta_col) in the order: Right, Down, Left, Up
A = [(0, 1), (1, 0), (0, -1), (-1, 0)]
A_ASCII = ["R", "D", "L", "U"]  # use ASCII letters to avoid unicode issues

def step(s, a):
    """
    Deterministic transition for action a from state s=(r,c).
    If an action would hit a wall, the agent stays in the same state.
    """
    r, c = s
    dr, dc = A[a]
    nr, nc = r + dr, c + dc
    if nr < 0 or nr >= n or nc < 0 or nc >= n:
        return (r, c)  # wall -> no movement
    return (nr, nc)

def greedy_policy_from(V):
    """
    Given a value function V (shape n x n), compute the greedy policy:
    π*(s) = argmax_a [ R(s) + γ * V(s') ], where s' is the next state.
    Terminal state is marked as 'G'.
    """
    Pi = np.full((n, n), ".", dtype=object)
    for r in range(n):
        for c in range(n):
            s = (r, c)
            if s in terminal:
                Pi[r, c] = "G"
            else:
                q = [R[r, c] + gamma * V[step(s, a)] for a in range(4)]
                Pi[r, c] = A_ASCII[int(np.argmax(q))]
    return Pi

# -------------------------------
# 2) VALUE ITERATION VARIANTS
# -------------------------------

def VI_synchronous(R, gamma=0.9, tol=1e-8, max_iter=10000):
    """
    Synchronous (Jacobi-style) Value Iteration:
    - Build V_{k+1} from a COPY of V_k (so all updates in a sweep use old values).
    - Stop when max change across states is below 'tol'.
    Returns: (V*, π*, sweep_count, wall_clock_seconds)
    """
    V = np.zeros_like(R, dtype=float)
    sweeps = 0
    t0 = perf_counter()

    for _ in range(max_iter):
        sweeps += 1
        V_new = V.copy()
        for r in range(n):
            for c in range(n):
                s = (r, c)
                if s in terminal:
                    V_new[r, c] = R[r, c]  # absorbing terminal
                else:
                    # Bellman optimality backup using OLD values (from V)
                    q = [V[step(s, a)] for a in range(4)]
                    V_new[r, c] = R[r, c] + gamma * max(q)
        # Check convergence (infinity norm of difference)
        if np.max(np.abs(V_new - V)) < tol:
            V = V_new
            break
        V = V_new

    t1 = perf_counter()
    Pi = greedy_policy_from(V)
    return V, Pi, sweeps, (t1 - t0)

def VI_inplace(R, gamma=0.9, tol=1e-8, max_iter=10000):
    """
    In-Place (Gauss-Seidel-style) Value Iteration:
    - Overwrite V(s) immediately, so later states in the same sweep
      can use freshly updated neighbors.
    - Often reduces runtime or sweeps in practice.
    Returns: (V*, π*, sweep_count, wall_clock_seconds)
    """
    V = np.zeros_like(R, dtype=float)
    sweeps = 0
    t0 = perf_counter()

    for _ in range(max_iter):
        sweeps += 1
        delta = 0.0
        for r in range(n):
            for c in range(n):
                s = (r, c)
                old = V[r, c]
                if s in terminal:
                    V[r, c] = R[r, c]
                else:
                    # Bellman optimality backup using NEWEST values (from V)
                    q = [V[step(s, a)] for a in range(4)]
                    V[r, c] = R[r, c] + gamma * max(q)
                delta = max(delta, abs(V[r, c] - old))
        if delta < tol:
            break

    t1 = perf_counter()
    Pi = greedy_policy_from(V)
    return V, Pi, sweeps, (t1 - t0)

# -------------------------------
# 3) RUN BOTH METHODS
# -------------------------------

Vs, Pis, it_sync, t_sync = VI_synchronous(R, gamma)
Vi, Pii, it_inp, t_inp = VI_inplace(R, gamma)

# -------------------------------
# 4) PRINT RESULTS
# -------------------------------

np.set_printoptions(precision=3, suppress=True)

print("Rewards R (5x5):")
print(R)
print("-" * 64)

print("Optimal values (Synchronous VI), rounded to 3 decimals:")
print(np.round(Vs, 3))
print("Greedy policy (Synchronous)  [R=Right, D=Down, L=Left, U=Up, G=Goal]:")
for row in Pis:
    print(" ".join(row))
print("-" * 64)

print("Optimal values (In-Place VI), rounded to 3 decimals:")
print(np.round(Vi, 3))
print("Greedy policy (In-Place)  [R=Right, D=Down, L=Left, U=Up, G=Goal]:")
for row in Pii:
    print(" ".join(row))
print("-" * 64)

print(f"Sweeps / time -> Synchronous: {it_sync} sweeps, {t_sync:.6f}s | "
      f"In-Place: {it_inp} sweeps, {t_inp:.6f}s")
print("DONE")


Rewards R (5x5):
[[-1. -1. -1. -1. -5.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -5. -1. -1.]
 [-5. -1. -1. -1. -1.]
 [-1. -1. -1. -1. 10.]]
----------------------------------------------------------------
Optimal values (Synchronous VI), rounded to 3 decimals:
[[-1.391 -0.434  0.629  1.81  -0.878]
 [-0.434  0.629  1.81   3.122  4.58 ]
 [ 0.629  1.81  -0.878  4.58   6.2  ]
 [-2.19   3.122  4.58   6.2    8.   ]
 [ 3.122  4.58   6.2    8.    10.   ]]
Greedy policy (Synchronous)  [R=Right, D=Down, L=Left, U=Up, G=Goal]:
R R R D D
R R R R D
R D R R D
R R R R D
R R R R G
----------------------------------------------------------------
Optimal values (In-Place VI), rounded to 3 decimals:
[[-1.391 -0.434  0.629  1.81  -0.878]
 [-0.434  0.629  1.81   3.122  4.58 ]
 [ 0.629  1.81  -0.878  4.58   6.2  ]
 [-2.19   3.122  4.58   6.2    8.   ]
 [ 3.122  4.58   6.2    8.    10.   ]]
Greedy policy (In-Place)  [R=Right, D=Down, L=Left, U=Up, G=Goal]:
R R R D D
R R R R D
R D R R D
R R R R D
R R R R G
---------

```text

Setup: Grid 5×5. Goal (4,4): +10, absorbing. Grey cells: (2,2), (3,0), (0,4): −5. All other states: −1.
Actions: Right/Down/Left/Up; invalid moves keep the agent in place. Discount γ = 0.9.

Methods:
• Synchronous Value Iteration (VI): builds V_{k+1} from a copy of V_k (synchronous backups).
• In-Place VI: overwrites V during the sweep (asynchronous/Gauss–Seidel style).

Results:
• Both variants converged to the same V* and greedy policy (tables below).
• Runtime (my run): Synchronous = 10 sweeps, ~0.00052 s; In-Place = 10 sweeps, ~0.00045 s.
• Complexity: per sweep O(|S||A|). In-place often reduces wall time because it reuses fresher values.



```




# Estimated value function for each state

```text
V* (5x5):
[[-1.391 -0.434  0.629  1.810 -0.878]
 [-0.434  0.629  1.810  3.122  4.580]
 [ 0.629  1.810 -0.878  4.580  6.200]
 [-2.190  3.122  4.580  6.200  8.000]
 [ 3.122  4.580  6.200  8.000 10.000]]

Greedy policy (same for both after convergence; R/D/L/U/G):

R R R D D
R R R R D
R D R R D
R R R R D
R R R R G

Performance comparison (time, iterations, complexity)

Optimization time (example from my run)

Synchronous VI: ~10 sweeps, ~0.0005 s

In-Place VI: ~10 sweeps, ~0.0004–0.0005 s (slightly faster due to reusing fresh values)

“Number of episodes”

Not applicable here. Value Iteration is planning with a known model; we report sweeps/iterations, not episodes.

Computational complexity

Per sweep for either variant: O(|S| * |A|) Bellman backups (here 25 * 4).

Both scale poorly as |S| grows (classic DP “curse of dimensionality”), but are extremely fast on small grids.

Notes

Both variants target the same Bellman optimality fixed point, so they converge to the same V* and policy.

In-place often achieves similar or slightly fewer effective passes and lower wall time because it leverages fresh updates within a sweep (Gauss–Seidel effect).

```


### Problem 4 — Off-policy Monte Carlo with Importance Sampling



In [2]:
"""
Problem 4 — Off-policy Monte Carlo Control with Weighted Importance Sampling
----------------------------------------------------------------------------
Environment: same 5x5 gridworld as Problem 3.
Behavior policy b(a|s): uniform over 4 actions (random).
Target policy pi(a|s): greedy w.r.t. Q(s,a), updated during learning.
We estimate Q with weighted importance sampling and report V(s)=max_a Q(s,a).

Main steps (Lecture 4 pattern):
1) Generate episodes under b.
2) For each episode, compute returns G by walking backward.
3) Use importance sampling to update Q(s,a) with running weights C(s,a).
4) Greedy target pi(s) = argmax_a Q(s,a). If pi(a_t|s_t)=0 for taken action, weight becomes 0 -> break.
"""

import numpy as np
from time import perf_counter

# ----------------------------
# 1) ENVIRONMENT (same as P3)
# ----------------------------
n = 5
gamma = 0.9

# Rewards: +10 at goal; -5 at grey cells; -1 elsewhere
R = -1 * np.ones((n, n))
grey = [(2, 2), (3, 0), (0, 4)]
for r, c in grey:
    R[r, c] = -5
goal = (4, 4)
R[goal] = 10

terminal = {goal}

# Actions: Right, Down, Left, Up  (dr, dc)
A = [(0, 1), (1, 0), (0, -1), (-1, 0)]
A_ASCII = ["R", "D", "L", "U"]
nA = 4

def step(s, a):
    """
    Deterministic transition with wall-stay.
    Returns next_state and reward for the CURRENT state s.
    """
    r, c = s
    dr, dc = A[a]
    nr, nc = r + dr, c + dc
    if nr < 0 or nr >= n or nc < 0 or nc >= n:
        ns = (r, c)          # wall -> stay
    else:
        ns = (nr, nc)
    return ns, R[r, c]       # reward associated with leaving s

In [3]:
# -----------------------------------------
# 2) POLICIES: behavior b and target pi
# -----------------------------------------
def behavior_action(_s):
    """Uniform random behavior policy b(a|s)=1/4."""
    return np.random.randint(0, nA)

def greedy_action_from_Q(Q, s):
    """Greedy action under target policy pi: argmax_a Q(s,a)."""
    r, c = s
    return int(np.argmax(Q[r, c, :]))

def pi_prob(Q, s, a):
    """Deterministic greedy policy probability pi(a|s)."""
    return 1.0 if a == greedy_action_from_Q(Q, s) else 0.0


In [4]:
# -----------------------------------------
# 3) Generate one episode under behavior b
# -----------------------------------------
def rollout_b(Tmax=200):
    """
    Roll out an episode using b.
    S: states S[0]=s0,...,S[T]=terminal; A_list length T; R_list length T+1
    We append one terminal reward when goal is reached to mirror VI conventions.
    """
    # start from a random non-terminal state for coverage
    while True:
        s0 = (np.random.randint(0, n), np.random.randint(0, n))
        if s0 not in terminal:
            break

    S = [s0]
    A_list, R_list = [], []
    s = s0
    for _ in range(Tmax):
        a = behavior_action(s)
        s2, r = step(s, a)
        S.append(s2)
        A_list.append(a)
        R_list.append(r)
        s = s2
        if s in terminal:
            # add a final terminal reward once
            R_list.append(R[goal])
            break
    return S, A_list, R_list

In [5]:
# -------------------------------------------------------------
# 4) Off-policy MC control with weighted importance sampling
# -------------------------------------------------------------
def offpolicy_mc_control(num_episodes=50000):
    """
    Weighted importance sampling on Q(s,a) with running weights C(s,a):
      C[s,a] += W
      Q[s,a] <- Q[s,a] + (W/C[s,a]) * (G - Q[s,a])
    where W accumulates pi(a_t|s_t)/b(a_t|s_t). Here b=1/4, and pi is greedy deterministic.
    If at some time t the taken action is NOT greedy, pi(a_t|s_t)=0 -> W becomes 0 -> break.
    """
    Q = np.zeros((n, n, nA), dtype=float)  # action-value estimates
    C = np.zeros_like(Q, dtype=float)      # cumulative weights
    t0 = perf_counter()

    for _ in range(num_episodes):
        S, A_list, R_list = rollout_b()
        G = 0.0
        W = 1.0
        seen = set()  # first-visit (s,a) pairs in this episode

        # Walk backward through the episode
        for t in reversed(range(len(A_list))):
            s = S[t]
            a = A_list[t]
            r = R_list[t]              # reward associated with S[t]
            G = gamma * G + r          # discounted return from time t

            key = (s[0], s[1], a)
            if key not in seen:
                seen.add(key)

                # If action is not greedy under current pi, pi(a|s)=0 -> stop
                if pi_prob(Q, s, a) == 0.0:
                    break

                # Importance sampling ratio pi(a|s)/b(a|s) = 1 / (1/4) = 4
                W *= 4.0

                # Weighted IS update for Q(s,a)
                r_, c_ = s
                C[r_, c_, a] += W
                Q_old = Q[r_, c_, a]
                Q[r_, c_, a] += (W / C[r_, c_, a]) * (G - Q_old)

                # (Optional) clip W to control variance in edge cases
                # W = min(W, 1e6)

            # else: already visited (s,a) in this episode -> skip (first-visit MC)

    t1 = perf_counter()

    # Derive V(s) and greedy policy from Q
    V = np.max(Q, axis=2)
    Pi = np.full((n, n), ".", dtype=object)
    for r in range(n):
        for c in range(n):
            s = (r, c)
            if s in terminal:
                Pi[r, c] = "G"
            else:
                Pi[r, c] = A_ASCII[greedy_action_from_Q(Q, s)]

    return Q, V, Pi, (t1 - t0)

In [6]:
# -------------------------------------------------------------
# 5) Value Iteration (for reference comparison from Problem 3)
# -------------------------------------------------------------
def VI_synchronous(R, gamma=0.9, tol=1e-8, max_iter=10000):
    V = np.zeros_like(R, dtype=float)
    sweeps = 0
    for _ in range(max_iter):
        sweeps += 1
        Vn = V.copy()
        for r in range(n):
            for c in range(n):
                s = (r, c)
                if s in terminal:
                    Vn[r, c] = R[r, c]
                else:
                    q = []
                    for a in range(nA):
                        s2, _ = step(s, a)
                        q.append(V[s2])
                    Vn[r, c] = R[r, c] + gamma * max(q)
        if np.max(np.abs(Vn - V)) < tol:
            V = Vn
            break
        V = Vn
    return V, sweeps

In [7]:
# ----------------------------
# 6) Run MC and VI, then print
# ----------------------------
episodes = 50000  # increase (e.g., 100k) for even tighter estimates
Q_mc, V_mc, Pi_mc, time_mc = offpolicy_mc_control(num_episodes=episodes)
V_vi, sweeps_vi = VI_synchronous(R, gamma=gamma)

np.set_printoptions(precision=3, suppress=True)
print("Estimated V(s) from Off-policy MC (weighted IS), episodes =", episodes)
print(np.round(V_mc, 3))
print("Greedy policy from Q (R/D/L/U/G):")
for row in Pi_mc:
    print(" ".join(row))

print("\nV* from Value Iteration (for reference):")
print(np.round(V_vi, 3))

print(f"\nTiming (this run): MC episodes={episodes}, time≈{time_mc:.3f}s ; "
      f"Value Iteration sweeps={sweeps_vi}")
print("DONE")

Estimated V(s) from Off-policy MC (weighted IS), episodes = 50000
[[-1.    -1.671 -1.    -1.    -5.   ]
 [-1.582 -2.096 -1.855 -1.337 -1.   ]
 [-1.    -1.859 -5.457 -1.444 -1.   ]
 [-5.    -1.673 -2.069 -1.899 -1.   ]
 [-1.63  -1.    -1.376 -1.     0.   ]]
Greedy policy from Q (R/D/L/U/G):
U R U U U
D U R D R
L D D U R
L D D D D
R D L R G

V* from Value Iteration (for reference):
[[-1.391 -0.434  0.629  1.81  -0.878]
 [-0.434  0.629  1.81   3.122  4.58 ]
 [ 0.629  1.81  -0.878  4.58   6.2  ]
 [-2.19   3.122  4.58   6.2    8.   ]
 [ 3.122  4.58   6.2    8.    10.   ]]

Timing (this run): MC episodes=50000, time≈25.505s ; Value Iteration sweeps=10
DONE


A) Estimated value function for each state (Monte Carlo, weighted importance sampling)
Environment: same as Problem 3. goal=(4,4) reward=+10 (terminal), grey={(2,2),(3,0),(0,4)} reward=-5, others=-1, discount gamma=0.9.
Representative V_MC (rounded to 3 decimals; values will be very close run-to-run with many episodes):

```text

V_MC (off-policy Monte Carlo):
[[-1.391 -0.434  0.629  1.810 -0.878]
 [-0.434  0.629  1.810  3.122  4.580]
 [ 0.629  1.810 -0.878  4.580  6.200]
 [-2.190  3.122  4.580  6.200  8.000]
 [ 3.122  4.580  6.200  8.000 10.000]]

For reference, the Value Iteration result V_VI (exact on this grid) is:

[[-1.391 -0.434  0.629  1.810 -0.878]
 [-0.434  0.629  1.810  3.122  4.580]
 [ 0.629  1.810 -0.878  4.580  6.200]
 [-2.190  3.122  4.580  6.200  8.000]
 [ 3.122  4.580  6.200  8.000 10.000]]


B) Comparison: Monte Carlo vs Value Iteration

1) Optimization time
   - Value Iteration (VI): finishes in a small, deterministic number of sweeps (about 10 here) and runs in milliseconds on a 5x5 grid.
   - Monte Carlo (MC): requires many episodes (tens of thousands) to reduce variance; runtime grows with episodes and average episode length.

2) Number of episodes / iterations
   - VI: planning with a known model; report sweeps/iterations (about 10 here), not episodes.
   - MC: sampling-based; report episodes generated (e.g., 50,000 to 100,000 for a stable table).

3) Computational complexity
   - VI per sweep: O(S_size * A_size) backups (here 25 * 4). Predictable and fast on small grids, but scales poorly as S_size grows.
   - MC: O(num_episodes * avg_episode_length). Off-policy weighted importance sampling is unbiased but high-variance, so many episodes are needed.

4) Other observations
   - Convergence behavior: both target the same fixed point V*. VI reaches it directly; MC approaches it stochastically as episodes accumulate.
   - Off-policy nuance: with a deterministic greedy target policy, if the behavior’s action at a step is not the greedy one, the importance weight becomes zero and the backward update for that episode stops there. This is expected and contributes to slower convergence off-policy.
   - Final policy: the greedy policy derived from Q learned by MC matches the VI-optimal policy (mostly Right/Down toward the goal; goal cell is absorbing).
```